<a href="https://colab.research.google.com/github/devasworski/Deeper-Networks-for-Image-Classification/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogleNet for Image Classification

Using GoogleNet on the MNIST and CIFAR datasets for image classification.

For the model definition, the code from [Khuyen Le](https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c) was used as a basis and then adjusted using the Guide from [Jai Faizy ](https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/) and the architecture of the Model was then updated to [V2](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202) 

## Imports

In [1]:
from py import googLeNet
from py import datasets
import os
from tensorflow.keras.callbacks import ModelCheckpoint

## Load Mnist

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = datasets.getMnist()

## Load Cifar10

In [2]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = datasets.getCifar()

## Load Model

In [3]:
model = googLeNet.googlenet(10)
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2022-04-30 22:58:15.549175: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                          

## Train

In [4]:
checkpoint_path = "/checkpoints/GoogLeNet/cp-{epoch:04d}.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path,verbose=1)

model.fit(x=x_train, y=y_train, epochs=20, verbose=1, validation_data=(x_test, y_test), batch_size=128, callbacks=cp_callback)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy_1/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/Users/asworski/opt/miniconda3/envs/DLCV/lib/python3